In [2]:
import pandas as pd
from sklearn.metrics import jaccard_score, pairwise_distances
import numpy as np

In [3]:
users=["User1", "User2", "User3","User4", "User5"]
items=["Item A", "Item B", "Item C", "Item D", "Item E"]
data=[
    [1, 0, 1, 1, 0],
    [1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 1],
    [1, 1, 1, 0, 1]
]
df = pd.DataFrame(data, columns = items, index = users)
df

,Item A,Item B,Item C,Item D,Item E
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


In [4]:
df.T

,User1,User2,User3,User4,User5
Item A,1,1,1,0,1
Item B,0,0,0,1,1
Item C,1,0,1,0,1
Item D,1,1,0,1,0
Item E,0,1,0,1,1


##### 直接计算两项的系数

In [5]:
jaccard_score(df['Item A'], df['Item B'])

0.2

In [6]:
jaccard_score(df['Item B'], df['Item C'])

0.25

##### 所有数据两两的系数

In [7]:
user_similarity = 1 - pairwise_distances(df.values, metric='jaccard')
user_similarity = pd.DataFrame(user_similarity, columns=users, index = users)
user_similarity

/Users/zhuoyujiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


,User1,User2,User3,User4,User5
User1,1.000000,0.50,0.666667,0.2,0.4
User2,0.500000,1.00,0.250000,0.5,0.4
User3,0.666667,0.25,1.000000,0.0,0.5
User4,0.200000,0.50,0.000000,1.0,0.4
User5,0.400000,0.40,0.500000,0.4,1.0


###### 计算物品相似度

In [61]:
item_similarity = 1- pairwise_distances(df.values.T, metric = 'jaccard')
item_similarity = pd.DataFrame(item_similarity, columns=items, index = items)
item_similarity['Item A']

/Users/zhuoyujiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Item A    1.00
Item B    0.20
Item C    0.75
Item D    0.40
Item E    0.40
Name: Item A, dtype: float64

In [9]:
topN_user={}
for i in user_similarity.index:
    # 取出每一列数据，并删除自身，然后排序
    column_after_sort=sorted(user_similarity.loc[i].drop([i]), reverse=True)[:2]
    topN_user[i] =column_after_sort
topN_user

{'User1': [0.6666666666666667, 0.5],
 'User2': [0.5, 0.5],
 'User3': [0.6666666666666667, 0.5],
 'User4': [0.5, 0.4],
 'User5': [0.5, 0.4]}

In [10]:
# aim= [0.5, 0.4]
# have = user_similarity.loc[:, 'User1'].to_list()
# index_num=[have.index(x) for x in aim]
#
# index = [user_similarity.index[i] for i in index_num]
# #df.T['User1'].replace(0, np.nan).dropna().index
# index+['User1']

In [11]:
# #list(df.T['User1'].replace(0, np.nan).dropna().index)
# similar_user =[]
# for x, y in user_similarity.items:
#     for each in y:
#         if user_similarity.loc[:, x] == each:
#             similar_user.append()
#         print(x)

In [12]:
# for values in topN_user:
#    # print(key)
#     print(values)

In [13]:
# result_dict={}
# for x in topN_user:
#    # print(x)
#    # similarity_num = user_similarity.loc[:, x].to_list()
#     user_similarity.loc[user_similarity[x] ==topN_user[x],]
#    # for y in similarity_num:
#    #     if y not in topN_user[x]:
#    #         result = user_similarity[x].replace(y, np.nan).dropna()
#     #    result_index = result.index
#     print(user_similarity)
#    # result_dict[x] = result_index
# #result_dict

In [14]:
# 构建推荐用户结果
recommend_result = {}
for x, y in topN_user.items():
    index_result = user_similarity.loc[user_similarity[x].isin(y)].index.tolist()
    recommend_result[x] = index_result
recommend_result

{'User1': ['User2', 'User3'],
 'User2': ['User1', 'User4'],
 'User3': ['User1', 'User5'],
 'User4': ['User2', 'User5'],
 'User5': ['User1', 'User2', 'User3', 'User4']}

In [47]:
# 协同推荐用户
user_filtered={}
user_set=set()
for user, recommend_users in recommend_result.items():
    for each in recommend_users:
        user_set = user_set.union(set(recommend_result[each]))
        user_set -= set(recommend_users)
        user_filtered[user] = list(user_set-set(user))
    user_filtered[user].remove(user)
user_filtered

{'User1': ['User4', 'User5'],
 'User2': ['User3', 'User5'],
 'User3': ['User4', 'User2'],
 'User4': ['User3', 'User1'],
 'User5': []}

In [53]:
# 过滤掉已有的推荐
item_filtered={}
item_set = set()
for key, value in user_filtered.items():
    remove_set = set(df.T[key].replace(0, np.nan).dropna().index)
    for u in value:
        item_set = item_set.union(set(df.T[u].replace(0, np.nan).dropna().index))
        item_set -= remove_set
        item_filtered[key] = list(item_set)
item_filtered

{'User1': ['Item E', 'Item B'],
 'User2': ['Item C', 'Item B'],
 'User3': ['Item E', 'Item D', 'Item B'],
 'User4': ['Item A', 'Item C']}

##### continuous data eg.rating

In [57]:
data2=[
    [5,3,4,4,None],
    [3,1,2,3,3],
    [4,3,4,3,5],
    [3,3,1,5,4],
    [1,5,5,2,1]
]
df2=pd.DataFrame(data2, columns=items, index=users)
user_similar = round(df2.T.corr(),4)
user_similar
# 用户2对某一电影的评分可以根据和他最相似的（用户1评分*相似度+用户4评分*相似度）/相似度之和

,User1,User2,User3,User4,User5
User1,1.0000,0.8528,0.7071,0.0000,-0.7921
User2,0.8528,1.0000,0.4677,0.4900,-0.9001
User3,0.7071,0.4677,1.0000,-0.1612,-0.4666
User4,0.0000,0.4900,-0.1612,1.0000,-0.6415
User5,-0.7921,-0.9001,-0.4666,-0.6415,1.0000


In [56]:
item_similar = round(df2.corr(),4)

##### ALS矩阵分解

###### 加权正则化交替最小二乘法

###### 将用户对商品的评分矩阵分解为两个矩阵：一个是用户对商品隐藏特征的偏好矩阵，一个是商品所包含的隐含特征的矩阵。在矩阵分解的过程中，评分缺失项得到了弥补，可以基于填充评分做用户推荐（使用als优化损失）

In [62]:
from zoneinfo._common import load_data


def predict(uid, iid, rating_matrix, user_matrix):

    """
    :param uid: 用户id
    :param iid: 物品id
    :param rating_matrix: 用户—物品评分矩阵
    :param user_matrix:  用户-用户相似矩阵
    :return: 预测的分数
    """

    print('开始预测用户<%d>对电影<%d>的评分...'%(uid, iid))
    # 1.找出uid用户的相似用户
    # 去掉用户自身
    similar_users = user_similar[uid].drop([uid]).dropna()
    # 相似用户筛选规则：正相关用户
    similar_users = similar_users.where(similar_users > 0).dropna()
    if similar_users.empty is True:
        raise Exception('用户<%d>没有相似的用户' % uid)

    # 2. 从uid用户的近邻用户中筛选出对iid物品有评分记录的近邻用户
    ids = set(rating_matrix[iid].dropna().index) & set(similar_users.index)
    final_similar_users = similar_users.loc[list(ids)]

    # 3. 结合uid用户与其临近相似用户的相似度预测uid用户对iid物品的评分
    sum_numerator = 0
    sum_denominator = 0
    for sum_uid, similarity in final_similar_users.iteritems():
        # 邻近用户的评分数据
        sum_user_rated_movies = rating_matrix.loc[sum_uid].dropna()
        # 邻近用户对iid物品的评分
        sim_user_rating_for_item = sum_user_rated_movies[iid]
        # 计算分子值
        sum_numerator += sim_user_rating_for_item
        # 计算分母值
        sum_denominator += similarity

    # 计算预测值并返回
    predict_rating = sum_numerator/sum_denominator
    print('预测用户<%d>对电影<%d>的评分：%0.2f' % (uid, iid, predict_rating))
    return round(predict_rating, 2)
if __name__ == '__main__':
    rating_matrix = load_data(DATA_PATH)
    user_similar = compute_person_similarity(rating_matrix, based = 'user')
    predict(1, 1, rating_matrix, user_similar)
    predict(1, 2, rating_matrix, user_similar)

In [ ]:
def predict_all(uid, ratings_matrix, user_similar):


In [ ]:
reading_dict ={'session'=session_list,'aid'=aid_list, 'type' = type_list}
reading_df = pd.DataFrame(reading_dict)
reading_matrix = reading_df.pivot_table(index = ['session'], columns=['aid'], values='type')

# 计算jaccard相关，得到用户之间相关系数 -> similarity（如果是连续型reading_matrix.T.corr()

# 预测用户1对物品1的评分
similar_users = similarity[1].drop([1]).dropna()
#找到正相关的用户
similar_users = similar_users.where(similar_users>0).dropna()
# 在所有正相关用户中找到为物品1消费过的用户
ids = reading_matrix[1].dropna().index & similar_users.index
final_similar_user = similar_users.loc[list(ids)]

# 计算用户1对物品1的评分
sum_numerator = 0
sum_denominator = 0
for sum_uid, similarity in final_similar_user.iteritems():
    # 邻近用户的评分数据
    sum_user_rated_movies = rating_matrix.loc[sum_uid].dropna()
    # 邻近用户对iid物品的评分
    sim_user_rating_for_item = sum_user_rated_movies[iid]
    # 计算分子值
    sum_numerator += sim_user_rating_for_item
    # 计算分母值
    sum_denominator += similarity
predict_rating = sum_numerator/sum_denominator



##### 梯度下降优化baseline损失

In [ ]:
# 数据加载
dtype = [('userId', np.int32), ('movie', np.int32), ('rating', np.float32)]
dataset = pd.read_csv('ratings.csv', usecols = range(3), dtype = dict(type))
list(dataset.itertuples())

In [ ]:
user_rating = dataset.groupby('userId').agg([list])
item_rating = dataset.groupby('itemId').agg([list])

In [ ]:
# 计算全局平均分
global_mean = dataset['rating'].mean()

In [ ]:
# 初始化bu, bi
bu = dict(zip(self.users_ratings.index, np.zero(len(self.users_ratings))))
bi = dict(zip(self.users_ratings.index, np.zero(len(self.users_ratings))))

for i in range(10):
    print('iter%d' % i) # 迭代次数
    for uid, iid, real_rating in dataset.itertuples(index = False):
        error = real_rating - (global_mean + bu[uid] +bi[uid])
        bu[uid] += 0.1*(error - 0.1 * bu[uid])
        bi[uid] += 0.1*(error - 0.1 * bu[iid])

In [ ]:
# 预测评分
def predict(uid, iid):
    predict_rating = global_mean + bu[uid] + bi[iid]
    return predict_rating

In [ ]:
# 数据集的拆分